In [ ]:
!pip install openai

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "발급받은 API 키"

In [ ]:
import sqlite3

# 데이터베이스 접속 생성
conn = sqlite3.connect('user_support.db')

# 커서 객체 생성
cursor = conn.cursor()

# users 테이블 생성
cursor.execute('''
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    phone TEXT
)
''')

# products 테이블 생성
cursor.execute('''
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    price INTEGER
)
''')

# order_history 테이블 생성
cursor.execute('''
CREATE TABLE order_history (
    history_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    product_id INTEGER,
    purchase_date DATE,
    quantity INTEGER,
    remarks TEXT,
    created_at DATETIME,
    FOREIGN KEY (user_id) REFERENCES users(user_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
''')

# support_history 테이블 생성
cursor.execute('''
CREATE TABLE support_history (
    history_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    subject TEXT,
    message_content TEXT,
    message_type TEXT,
    status TEXT,
    created_at DATETIME,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
)
''')

# 변경 사항 커밋
conn.commit()

# 데이터베이스 접속 종료
conn.close()

In [ ]:
import sqlite3
from datetime import datetime

# 데이터베이스 접속 생성
conn = sqlite3.connect('user_support.db')

# 커서 객체 생성
cursor = conn.cursor()

# users 테이블에 한국어 테스트 데이터 삽입
users_data = [
    (1, '태진', '윤', 'taejin@example.com', '099-1234-5678'),
    (2, '은미', '김', 'eunmi@example.com', '098-9876-5432')
]

for user in users_data:
    cursor.execute('''
    INSERT INTO users (user_id, first_name, last_name, email, phone)
    VALUES (?, ?, ?, ?, ?)
    ''', user)

# products 테이블에 한국어 테스트 데이터 삽입
products_data = [
    (1, '제품A', 10000),
    (2, '제품B', 20000),
    (3, '제품C', 30000)
]

for product in products_data:
    cursor.execute('''
    INSERT INTO products (product_id, product_name, price)
    VALUES (?, ?, ?)
    ''', product)

# order_history 테이블에 한국어 테스트 데이터 삽입
order_history_data = [
    (1, 1, 1, '2023-10-01', 2, '빠른 발송', datetime.now()),
    (2, 2, 3, '2023-10-05', 1, '선물 포장', datetime.now())
]

for order in order_history_data:
    cursor.execute('''
    INSERT INTO order_history (history_id, user_id, product_id, purchase_date, quantity, remarks, created_at)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', order)

# support_history 테이블에 한국어 테스트 데이터 삽입
support_history_data = [
    (1, 1, '청구 관련 문제', None, None, 'open', datetime.now()),
    (2, 1, None, '청구에 문제가 있습니다.', 'user', None, datetime.now()),
    (3, 1, None, '문의해주셔서 감사합니다. 문제를 조사하고 있습니다.', 'support', None, datetime.now())
]

for support in support_history_data:
    cursor.execute('''
    INSERT INTO support_history (history_id, user_id, subject, message_content, message_type, status, created_at)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', support)

# 변경 사항 커밋
conn.commit()

# 데이터베이스 접속 종료
conn.close()

In [ ]:
import sqlite3

# 데이터베이스 접속 생성
conn = sqlite3.connect('user_support.db')

# 커서 객체 생성
cursor = conn.cursor()

# 각 테이블에서 데이터를 얻어 표시
tables = ['users', 'products', 'order_history', 'support_history']

for table in tables:
    print(f"{table} 테이블의 데이터:")
    cursor.execute(f"SELECT * FROM {table}")
    rows = cursor.fetchall()

    for row in rows:
        print(row)

    print()  # 보기 쉽도록 빈 행 추가

# 데이터베이스 접속 종료
conn.close()

In [ ]:
from openai import OpenAI
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
from sqlalchemy.exc import SQLAlchemyError
from tabulate import tabulate
from json.decoder import JSONDecodeError
import json
import re

engine = create_engine('sqlite:///user_support.db', echo=False)

client = OpenAI()

DEBUG = False

# 기정의된 시스템 메시지
SYSTEM_MESSAGE = """
다음과 같이 정의된 테이블이 있습니다.
정의 데이터:
{meta_text}

이 정의 데이터를 바탕으로 사용자의 요청문에 가장 적합한 SQL 문을 생성합니다.
다음과 같은 JSON 형식으로만 출력해 주세요. 설명은 150자 이내로 작성해주세요.

{{
    "sql": (SQL 문),
    "description": (설명)
}}

## 대화 에시
USER->사용자 목록을 출력해 주세요.
AI->{{"sql" : "SELECT user_id, first_name, last_name, email, phone FROM users","description" : "사용자 목록을 얻기 위해, users 테이블에서 필요한 정보를 얻는 SELECT 문을 생성했습니다."}}
"""

def create_sql_response(text, meta_text):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature = 0.3,
        messages=[
            {
                "role": "system",
                "content": SYSTEM_MESSAGE.format(meta_text=meta_text)
            },
            {
                "role": "user",
                "content": f"요청문: {text}...결과는 JSON 형식으로 출력해주세요"
            }
        ],
    )
    try:
        if DEBUG:
            print(response.choices[0].message.content)
        json_str = re.search(r'\{.*\}', response.choices[0].message.content, re.DOTALL).group(0)
        return json.loads(json_str)
    except JSONDecodeError:
        return {
            "sql": None,
            "description": None
        }

def get_meta_data():
    metadata = MetaData()
    metadata.reflect(bind=engine)

    meta_text = ''
    for table in metadata.tables.values():
        meta_text += '테이블 이름: '+ table.name + '\n'
        for column in table.columns:
            meta_text += f'컬럼 이름: {column.name}, 형식: {column.type}\n'
        meta_text += '\n'
    return meta_text

def exec_sql(sql):
    Session = sessionmaker(bind=engine)

    try:
        with Session() as session:
            t = text (sql)
            result = session.execute(t)

            header = [k for k in result.keys()]

            rows = result.fetchall()
            tabled = tabulate(rows,header, tablefmt="github")
            print(tabled)

    except SQLAlchemyError as e:
        print(f'Exception Excute SQL: {e}\n')

def repl():
    meta_text = get_meta_data()
    if not meta_text:
        print("메타데이터가 입력되어 있지 않습니다")
        return
    print('메타데이터를 읽었습니다\n')
    while True:
        try:
          user_input = input(">")
          if user_input:
              response = create_sql_response(user_input, meta_text)
              sql = response['sql']
              description = response['description']

              print('SQL 문:\n', sql)
              print('설명:\n', description)
              print()
              if sql:
                exec_sql(sql)
        except (KeyboardInterrupt, EOFError):
            print()
            break

if __name__ == "__main__":
    repl()